<a href="https://colab.research.google.com/github/franlin1860/llm/blob/main/marker_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip3 install marker-pdf


In [44]:
!marker_single --help

2024-08-01 03:24:30.708283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 03:24:30.745673: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 03:24:30.757225: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 03:24:32.720165: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: marker_single [-h] [--max_pages MAX_PAGES] [--start_page START_PAGE] [--langs LANGS]
                     [--batch_multiplier BATCH_MULTIPLIER] [--debug]
                     filename output

positional arguments:
  filename          

In [42]:
!mkdir /output

In [43]:
!marker_single mjzf.pdf /output --batch_multiplier 2 --max_pages 1

2024-08-01 03:14:02.982195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 03:14:03.015033: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 03:14:03.030109: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-01 03:14:05.779999: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loaded detection model vikp/surya_det3 on device cpu with dtype torch.float32
Loaded detection model vikp/surya_layout3 on device cpu with dtype torch.float32
Loaded reading order model vikp/surya_order on device cpu with dtype torch.float32

In [48]:
!pip install llama-index


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.9 MB/s eta 0:00:00


In [49]:
!pip install llama-index-readers-pdf_marker

In [51]:
!pip install llama_index-embeddings-huggingface

In [50]:
!pip install llama_index-llms-openai_like

In [52]:
!pip install llama_index-readers-file

In [53]:
!pip install openai

In [54]:
!env DEEPSEEK_API_KEY="sk-4944cc4af34148f6a53f74244a48bf8b"

SHELL=/bin/bash
NV_LIBCUBLAS_VERSION=12.2.5.6-1
NVIDIA_VISIBLE_DEVICES=all
COLAB_JUPYTER_TRANSPORT=ipc
NV_NVML_DEV_VERSION=12.2.140-1
NV_CUDNN_PACKAGE_NAME=libcudnn8
CGROUP_MEMORY_EVENTS=/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
NV_LIBNCCL_DEV_PACKAGE=libnccl-dev=2.19.3-1+cuda12.2
NV_LIBNCCL_DEV_PACKAGE_VERSION=2.19.3-1
VM_GCE_METADATA_HOST=169.254.169.253
HOSTNAME=7bc0c6150b4c
LANGUAGE=en_US
TBE_RUNTIME_ADDR=172.28.0.1:8011
PYTORCH_ENABLE_MPS_FALLBACK=1
COLAB_TPU_1VM=
GCE_METADATA_TIMEOUT=3
NVIDIA_REQUIRE_CUDA=cuda>=12.2 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,driver<471 brand=geforcertx,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=titan,driver>=470,driver<471 brand=titanrtx,driver>=470,driver<471 brand=tesla,driver>=525,driver<526 brand=unknown,driver>=525,dri

In [62]:
import os
import logging
import sys
from llama_index.llms.openai_like import OpenAILike
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.file import PDFReader
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.readers.pdf_marker import PDFMarkerReader
from pathlib import Path
import dotenv

In [63]:
# 导入环境变量
dotenv.load_dotenv('./.env')

# 配置日志
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# 定义DeepSpeed model
llm = OpenAILike(model="deepseek-chat",
                 api_base="https://api.deepseek.com/v1",
                 api_key=os.environ["DEEPSEEK_API_KEY"],
                 temperature=0.6,
                 is_chat_model=True)

# print(os.environ["DEEPSEEK_API_KEY"])

# 配置环境
Settings.llm = llm

# 设置嵌入模型
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-zh-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 768

sk-4944cc4af34148f6a53f74244a48bf8b


In [64]:
response = llm.complete("Hello World!")
print(str(response))


Hello World! It's a pleasure to meet you. How can I assist you today?


In [73]:
from pathlib import Path
from llama_index.readers.pdf_marker import PDFMarkerReader

# PDF Reader with `pdf maker`
path = Path("./mjzf.pdf")
if not path.exists():
    print("文件不存在，程序即将退出")
    sys.exit()
reader = PDFMarkerReader()
documents = reader.load_data(file=path, max_pages=1, start_page=0)
documents = []
start_page = 0
max_pages = 13 # Replace with the actual number of pages in the PDF
page = 0

for page in range(start_page, max_pages):
    documents.extend(reader.load_data(file=path, max_pages=1, start_page=page))
    page += 1

Loaded detection model vikp/surya_det3 on device cpu with dtype torch.float32
Loaded detection model vikp/surya_layout3 on device cpu with dtype torch.float32
Loaded reading order model vikp/surya_order on device cpu with dtype torch.float32
Loaded recognition model vikp/surya_rec on device cpu with dtype torch.float32
Loaded texify model to cpu with torch.float32 dtype


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:510: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Finding reading order: 100%|██████████| 1/1 [00:17<00:00, 17.38s/it]


Loaded detection model vikp/surya_det3 on device cpu with dtype torch.float32
Loaded detection model vikp/surya_layout3 on device cpu with dtype torch.float32
Loaded reading order model vikp/surya_order on device cpu with dtype torch.float32
Loaded recognition model vikp/surya_rec on device cpu with dtype torch.float32
Loaded texify model to cpu with torch.float32 dtype


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:510: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Finding reading order: 100%|██████████| 1/1 [00:16<00:00, 16.87s/it]


Loaded detection model vikp/surya_det3 on device cpu with dtype torch.float32
Loaded detection model vikp/surya_layout3 on device cpu with dtype torch.float32
Loaded reading order model vikp/surya_order on device cpu with dtype torch.float32
Loaded recognition model vikp/surya_rec on device cpu with dtype torch.float32
Loaded texify model to cpu with torch.float32 dtype


...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:510: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Finding reading order: 100%|██████████| 1/1 [00:16<00:00, 16.05s/it]


Loaded detection model vikp/surya_det3 on device cpu with dtype torch.float32
Loaded detection model vikp/surya_layout3 on device cpu with dtype torch.float32
Loaded reading order model vikp/surya_order on device cpu with dtype torch.float32
Loaded recognition model vikp/surya_rec on device cpu with dtype torch.float32
Loaded texify model to cpu with torch.float32 dtype


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:510: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Finding reading order: 100%|██████████| 1/1 [00:15<00:00, 15.44s/it]


Loaded detection model vikp/surya_det3 on device cpu with dtype torch.float32
Loaded detection model vikp/surya_layout3 on device cpu with dtype torch.float32
Loaded reading order model vikp/surya_order on device cpu with dtype torch.float32
Loaded recognition model vikp/surya_rec on device cpu with dtype torch.float32
Loaded texify model to cpu with torch.float32 dtype


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/image_processing_auto.py:510: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(
Finding reading order: 100%|██████████| 1/1 [00:15<00:00, 15.78s/it]


In [74]:
index = VectorStoreIndex.from_documents(
    documents,
)


In [75]:
# 配置检索器
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)

# 配置响应合成器
response_synthesizer = get_response_synthesizer()

# 组装查询引擎
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [76]:
# 遍历并打印每个节点
for node in index.docstore.docs.values():
    print(node)

Node ID: 1eddc509-789e-4a5e-b474-42d0d034bd32
Text: # 泰康在线财产保险股份有限公司 门急诊费用医疗保险 C 款(互联网专属)条款
在本条款中,"您"指投保人,"我们"、"本公司"均指泰康在线财产保险股份有限公司。   | ❶
| |--------------------------------------------------------------| |
您与我们的合同                                               | | 1.1
| | 合同构成   ...
Node ID: 51816348-dd98-453a-94b8-5a132d5db45b
Text: 具  在等待期内,被保险人非因意外伤害(见 8.2)经医院(见 8.3)确
诊罹患疾病,在医院接受门(急)诊治疗或自医院确诊罹患疾病后  在我们指定的互联网医院(见 8.4)进行后续诊疗的,无论诊疗行为
是否发生在等待期内,我们均不承担赔偿保险金的责任,但本合同 继续有效。 您为被保险人续保本保险或被保险人因遭受意外伤害导致被保险人
(一)若被保险人未以参加社会基本医疗保险或者公费医疗身份就
Node ID: cae31f6b-06de-4461-9161-e7b5cbcefde4
Text: 医疗费用,在扣除单次门(急)诊医疗费用免赔额后,在单次门(急) 诊医疗费用保险金额内按照赔偿比例 A 进行赔偿;
(二)若被保险人以参加社会基本医疗保险或者公费医疗身份就诊 并结算的,我们对被保险人每次就诊发生的符合保险责任范围的医
疗费用,在扣除单次门(急)诊医疗费用免赔额后,在单次门(急) 诊医疗费用保险金额内按照赔偿比例 B 进行赔偿。
单次门(急)诊医疗费用免赔额、赔偿比例 A 及赔偿比例 B 由您与 我们协商确定,并在保险单中载明。
被保险人于同一日(当日零时起至二十四时止)在同一所医院同一 个科室的就诊视为一次就诊。
在本合同保险期间内,被保险人无论一次或多次在医院进行治疗, 我们均按上述约定承担赔偿责任。每次门(急)诊医疗费用保险金
赔偿以单次门(急)诊医疗费用保险...
Node ID: 6e907bb3-fc8d-47ce-9349-d61fa7039933
Text: 被保险人因遭受意外伤害事

In [77]:
# 查询
response = query_engine.query("互联网药店直付用药流程?")
print(response, "\n")


互联网药店直付用药流程涉及以下步骤：
1. 被保险人在指定的互联网医院接受诊疗并获得医生开具的药品处方。
2. 药品处方需符合中国国家药品监督管理部门批准的说明书所列明的适应症及用法用量，且剂量不超过一个月。
3. 药品必须是保险单载明的“互联网医院药品清单”中的药品，并在认可的互联网药店购买。
4. 对于符合条件的药品费用，保险公司将按照合同约定的赔偿比例进行赔偿。
5. 如果药品费用不符合上述任一条件，保险公司不承担赔偿责任。
6. 被保险人在同一日内由同一个医生诊疗并开具处方的视为一次就诊。
7. 保险期间内，无论被保险人进行多少次诊疗，每次赔偿金额以单次互联网医院药品费用保险金额为限，累计达到保险金额时，该项保险责任终止。 



In [78]:
response = query_engine.query("治疗期间发生的必需且合理的医疗费用包括什么？?")
print(response, "\n")

治疗期间发生的必需且合理的医疗费用包括护理费、检查检验费、诊疗费、治疗费、药品费和门诊手术费。具体来说，护理费是指在门（急）诊治疗期间根据医嘱所示的护理服务费用；检查检验费包括实际发生的以诊断为目的的医学手段检查及检验费用，如诊查费、妇检费、X光费、心电图费、B超费、脑电图费、内窥镜费、肺功能仪费、分子生化检验费和血、尿、便常规检验费等；诊疗费是指在门（急）诊治疗期间发生的经治医生的技术劳务费用；治疗费包括以治疗疾病为目的的医学手段产生的合理费用，如注射费、机疗费、理疗费、输血费、输氧费、体外反搏费等；药品费是指医生开具的具有国家药品监督管理部门核发的药品批准文号的药品费用；门诊手术费则是指在门诊进行的手术相关费用。 



In [79]:
response = query_engine.query("门急诊费用医疗保险C款的保险费约定支付日?")
print(response, "\n")

门急诊费用医疗保险C款的保险费约定支付日是保险合同生效日在每月的对应日。如果当月无对应的同一日，则以该月最后一日为准。 

